This is intended to be an example of the workflow / requirements to generate observations of a moving object. This shows the general workflow for generating observations using PyOrb to generate ephemerides (here, 'ra/dec' positions at an observatory). 

This also demonstrates the basic functionality which we need from ADAM, and could imagine wrapping ADAM in a class to make it easy to call (like we've wrapped PyOorb here, with lsst.sims.movingObjects.PyOrbEphemerides). 

The LSST sims_movingObject code is at https://github.com/lsst/sims_movingObjects

In [1]:
import numpy as np
import pandas as pd

# Import lsst sims_movingObjects code -- 
# this has some convenience classes for reading orbits and generating ephemerides with PyOrb
import lsst.sims.movingObjects as mo

In [2]:
# Example of an input file we typically use.
!head 'mba_300.s3m'

!!S3M V.09.06.15
!!OID FORMAT q e i node argperi t_p H t_0 INDEX N_PAR MOID COMPCODE
S1000000a  COM   3.01822   0.05208  22.56035 211.00286 335.42134  51575.94061 14.20  54800.00000 1 6 -1 Python
S1000001a  COM   2.10974   0.07518   4.91571 209.40298 322.66447  54205.77161 20.57  54800.00000 1 6 -1 Python
S1000002a  COM   2.80523   0.07777   1.24945 112.52284 139.86858  54468.71747 14.65  54800.00000 1 6 -1 Python
S1000003a  COM   2.10917   0.13219   1.46615 266.54621 232.24412  54212.16304 19.58  54800.00000 1 6 -1 Python
S1000004a  COM   2.17676   0.19949  12.92422 162.14580 192.22312  51895.46586 10.56  54800.00000 1 6 -1 Python
S1000007a  COM   1.98262   0.16324  23.42404 210.02222 326.18514  54730.97656  8.17  54800.00000 1 6 -1 Python
S1000008a  COM   1.73305   0.24339  23.73823 337.86076 113.29921  54647.98297  8.97  54800.00000 1 6 -1 Python
S1000009a  COM   2.18691   0.15787  14.26404  63.83526 171.52690  51866.56387 10.73  54800.00000 1 6 -1 Python


In [3]:
orbits = mo.Orbits()
orbits.readOrbits('mba_300.s3m')

In [4]:
# What is the data that is in orbits? a pandas data frame of orbital parameters, plus the format.
print(orbits.orb_format)
orbits.orbits[0:3]

COM


,objId,q,e,inc,Omega,argPeri,tPeri,H,epoch,g,sed_filename
0,S1000000a,3.01822,0.05208,22.56035,211.00286,335.42134,51575.94061,14.20,54800.0,0.15,C.dat
1,S1000001a,2.10974,0.07518,4.91571,209.40298,322.66447,54205.77161,20.57,54800.0,0.15,C.dat
2,S1000002a,2.80523,0.07777,1.24945,112.52284,139.86858,54468.71747,14.65,54800.0,0.15,S.dat


In [5]:
# Okay - set up ephemeris generation module. (Here this is PyOorb. Swap for ADAM)
ephGen = mo.PyOrbEphemerides(ephfile='de430')
ephGen.setOrbits(orbits)

In [6]:
# The pyoorb orbit saves a copy of the orbit elements, in the format that pyoorb needs
# Probably the ADAM module would turn the given elements into Cartesian format and store in the dictionary (OPM?)
ephGen.oorbElem[0]

array([  0.00000000e+00,   3.01822000e+00,   5.20800000e-02,
         3.93752388e-01,   3.68269464e+00,   5.85420676e+00,
         5.15759406e+04,   2.00000000e+00,   5.48000000e+04,
         3.00000000e+00,   1.42000000e+01,   1.50000000e-01])

In [7]:
# I want to be able to propagate orbits. 
# original epoch - just FYI
epoch_orig = orbits.orbits['epoch'][0]
epoch_new = epoch_orig + 30.0
print('Test things out to propagate from %f to %f' % (epoch_orig, epoch_new))
#ephGen.propagateOrbits(epoch_new)   --- currently doesn't work for PyOrb
# epoch_new could be a single time or a range of times would be useful too (numpy array?)
# New orbits stored in the ephGen object (i.e. series of OPM dictionaries?)

Test things out to propagate from 54800.000000 to 54830.000000


In [8]:
# And I want to be able to generate 'observations' at a series of given times and at a given observatory location
times = np.sort(epoch_orig + (np.random.rand(100)) * (epoch_new - epoch_orig))
print(times)

[ 54800.0272083   54800.11652983  54800.91350783  54801.39857434
  54801.5635174   54802.43987338  54802.79619456  54803.06728361
  54803.19663425  54803.40330753  54803.67261494  54804.0772348
  54804.5914872   54805.15884506  54805.21112618  54805.88469292
  54806.43759611  54806.5204512   54806.57061195  54806.86912816
  54806.95686385  54807.06634154  54807.31373372  54807.46442235
  54807.48513908  54807.63034795  54807.72443812  54807.84384013
  54807.89571734  54808.11334993  54808.4661842   54808.95994545
  54809.08319158  54809.15419397  54809.22838755  54809.67765201
  54810.7302052   54811.08729443  54811.40556941  54811.8397688
  54812.29820406  54812.35290797  54812.65272106  54812.99498542
  54813.24089402  54813.50772355  54813.72940532  54814.20614858
  54814.43621976  54814.68821595  54814.76430519  54815.26807562
  54815.62255328  54815.73107937  54816.31424347  54816.31812373
  54816.47845006  54817.47537262  54817.54634764  54817.75472913
  54817.84089316  54818.270

In [9]:
obs = ephGen.generateEphemerides(times=times, obscode='I11', byObject=True)

In [10]:
# The ephems returned are a 2d recarray, which can be grouped as either [object][time] or [time][object] 
# (how these are sorted is set by the 'byObject' flag)
print(obs.shape)
print(obs[0][0].dtype.names)
# e.g. here the observations were grouped by object - so these are of the same object, at various times.
obs[0]['time']

(300, 100)
('delta', 'ra', 'dec', 'magV', 'time', 'dradt', 'ddecdt', 'phase', 'solarelon', 'velocity')


array([ 54800.0272083 ,  54800.11652983,  54800.91350783,  54801.39857434,
        54801.5635174 ,  54802.43987338,  54802.79619456,  54803.06728361,
        54803.19663425,  54803.40330753,  54803.67261494,  54804.0772348 ,
        54804.5914872 ,  54805.15884506,  54805.21112618,  54805.88469292,
        54806.43759611,  54806.5204512 ,  54806.57061195,  54806.86912816,
        54806.95686385,  54807.06634154,  54807.31373372,  54807.46442235,
        54807.48513908,  54807.63034795,  54807.72443812,  54807.84384013,
        54807.89571734,  54808.11334993,  54808.4661842 ,  54808.95994545,
        54809.08319158,  54809.15419397,  54809.22838755,  54809.67765201,
        54810.7302052 ,  54811.08729443,  54811.40556941,  54811.8397688 ,
        54812.29820406,  54812.35290797,  54812.65272106,  54812.99498542,
        54813.24089402,  54813.50772355,  54813.72940532,  54814.20614858,
        54814.43621976,  54814.68821595,  54814.76430519,  54815.26807562,
        54815.62255328,  

In [11]:
# Here they are sorted in the other order, so it's all objects @ the first time, then the second time, etc.
obs = ephGen.generateEphemerides(times=times, obscode='I11', byObject=False)
print(obs.shape)
print(obs[0][0].dtype.names)
print(obs[0]['time'])

(100, 300)
('delta', 'ra', 'dec', 'magV', 'time', 'dradt', 'ddecdt', 'phase', 'solarelon', 'velocity')
[ 54800.0272083  54800.0272083  54800.0272083  54800.0272083  54800.0272083
  54800.0272083  54800.0272083  54800.0272083  54800.0272083  54800.0272083
  54800.0272083  54800.0272083  54800.0272083  54800.0272083  54800.0272083
  54800.0272083  54800.0272083  54800.0272083  54800.0272083  54800.0272083
  54800.0272083  54800.0272083  54800.0272083  54800.0272083  54800.0272083
  54800.0272083  54800.0272083  54800.0272083  54800.0272083  54800.0272083
  54800.0272083  54800.0272083  54800.0272083  54800.0272083  54800.0272083
  54800.0272083  54800.0272083  54800.0272083  54800.0272083  54800.0272083
  54800.0272083  54800.0272083  54800.0272083  54800.0272083  54800.0272083
  54800.0272083  54800.0272083  54800.0272083  54800.0272083  54800.0272083
  54800.0272083  54800.0272083  54800.0272083  54800.0272083  54800.0272083
  54800.0272083  54800.0272083  54800.0272083  54800.0272083 

In [12]:
# But I would be okay with just generating observations at a single time, if the propagation was still efficient
# (here for PyOorb, generating all the ephemerides at one time is more efficient than doing it per time, using the 
# orbits at the original epoch)

Here's a rough/simple example of our end-to-end process..

In [13]:
# Use a MAF class to quickly grab data from the opsim sqlite output. The result here is just a numpy recarray.
import lsst.sims.maf.db as db
o = db.OpsimDatabase('/Users/lynnej/opsim/db/astro-lsst-01_2013.db')
surveydata = o.fetchMetricData(colnames=['observationStartMJD', 'fieldRA', 'fieldDec', 
                                         'filter', 'seeingFwhmGeom', 'fiveSigmaDepth'],
                               sqlconstraint='night < 5')

In [14]:
import numbers
def angularSeparation(long1, lat1, long2, lat2):
    """
    Angular separation between two points in radians
    Parameters
    ----------
    long1 is the first longitudinal coordinate in radians
    lat1 is the first latitudinal coordinate in radians
    long2 is the second longitudinal coordinate in radians
    lat2 is the second latitudinal coordinate in radians
    Returns
    -------
    The angular separation between the two points in radians
    Calculated based on the haversine formula
    From http://en.wikipedia.org/wiki/Haversine_formula
    """
    t1 = np.sin(lat2/2.0 - lat1/2.0)**2
    t2 = np.cos(lat1)*np.cos(lat2)*np.sin(long2/2.0 - long1/2.0)**2
    ss = t1 + t2
    ss = np.where(ss<0.0, 0.0, ss)
    return 2.0*np.arcsin(np.sqrt(ss))


def ssoInCircleFov(ephems, obsData, rFov=2.1):
    """Determine which observations are within a circular fov for a series of observations.

    Parameters
    ----------
    ephems : np.recarray
        Ephemerides for the objects, with RA and Dec as 'ra' and 'dec' columns (in degrees).
    obsData : np.recarray
        The observation pointings, with RA and Dec as 'ra' and 'dec' columns (in degrees).
    rFov : float, opt
        The radius of the field of view, in degrees.
        Default 2.1 is appropriate for LSST fov if later applying camera footprint.
        A value of 1.75 would be appropriate for simple circular LSST FOV assumption.

    Returns
    -------
    np.ndarray
        Returns the indexes of the numpy array of the object observations which are inside the fov.
    """
    sep = angularSeparation(np.radians(ephems['ra']), np.radians(ephems['dec']), 
                            np.radians(obsData['fieldRA']), np.radians(obsData['fieldDec']))
    idxObs = np.where(sep <= np.radians(rFov))[0]
    return idxObs

In [15]:
def calcTrailingLosses(velocity, seeing, texp=30.):
    """Calculate the detection and SNR traiiling losses.

    Parameters
    ----------
    velocity : np.ndarray or float
        The velocity of the moving objects, in deg/day.
    seeing : np.ndarray or float
        The seeing of the images, in arcseconds.
    texp : np.ndarray or float, opt
        The exposure time of the images, in seconds. Default 30.

    Returns
    -------
    (np.ndarray, np.ndarray) or (float, float)
        dmagTrail and dmagDetect for each set of velocity/seeing/texp values.
    """
    a_trail = 0.761
    b_trail = 1.162
    a_det = 0.420
    b_det = 0.003
    x = velocity * texp / seeing / 24.0
    dmagTrail = 1.25 * np.log10(1 + a_trail * x ** 2 / (1 + b_trail * x))
    dmagDetect = 1.25 * np.log10(1 + a_det * x ** 2 / (1 + b_det * x))
    return (dmagTrail, dmagDetect)

In [16]:
#%%timeit 6.71 s ± 252 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# Timeit magic says this takes about 6.7s per loop, with 5 days of simulated data (2724 visits or 'times')
# and only 10 objects.
# and note that this is with some extra propagation - orbit epoch is 54800.00000 but 
# the first visit is at 59853.016793981478  (so there's an extra 13 years of propagation here .. whoops)

# Let's try it again with some orbits with a 'faked' epoch that matches t0
# Okay, then it's 4.92 s ± 58.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) .. so not so different.

# 5 days, 10 objects ~ 5-7s ... so for the full 10 years, this is about 5k sec (85 minutes) for 10 objects.
# We'd prefer to run for 2k-10k objects, most of the time (for survey completeness kind of stuff) = ~1500 hours
# but occasionally would like to run millions of objects for a shorter length of time (say about a month)
# Note that both of these cases are pretty trivially parallelizable


times = surveydata['observationStartMJD']
columns=['objId', 'ra', 'dec', 'delta', 'dradt', 'ddecdt', 'velocity', 'magV', 'phase', 'solarelon',
                           'observationStartMJD', 'fieldRA', 'fieldDec', 'filter', 'seeingFwhmGeom', 'fiveSigmaDepth',
                           'dmagTrail', 'dmagDetect', 'dmagColor']

surveyobs = None
for sso in orbits[0:10]:
    ephGen.setOrbits(sso)
    ephs = ephGen.generateEphemerides(times, obscode='I11', byObject=True)
    ephs = ephs[0]
    idxObs = ssoInCircleFov(ephs, surveydata, rFov=1.75)
    nobs = len(idxObs)
    #print('Found %d observations with the telescope for object %s' % (nobs, sso.orbits.objId.values[0]))
    if nobs > 0:
        obs = ephs[idxObs]
        metaobs = surveydata[idxObs]
        dmagTrail, dmagDetect = calcTrailingLosses(obs['velocity'], metaobs['seeingFwhmGeom'], 30.)
        dmagColor = np.zeros(nobs, float)
        d = {'objId': [sso.orbits.objId.values[0]]*nobs,
            'ra': obs['ra'], 'dec': obs['dec'], 'delta': obs['delta'],
             'dradt': obs['dradt'], 'ddecdt': obs['ddecdt'], 'velocity': obs['velocity'], 
             'magV': obs['magV'], 'phase': obs['phase'], 'solarelon': obs['solarelon'],
             'observationStartMJD': metaobs['observationStartMJD'], 'fieldRA': metaobs['fieldRA'],
             'fieldDec': metaobs['fieldDec'], 'filter': metaobs['filter'], 
             'seeingFwhmGeom': metaobs['seeingFwhmGeom'], 'fiveSigmaDepth': metaobs['fiveSigmaDepth'],
             'dmagTrail': dmagTrail, 'dmagDetect': dmagDetect, 'dmagColor': dmagColor}
        newobs = pd.DataFrame(d)
        if surveyobs is None:
            surveyobs = newobs
        else:
            surveyobs = surveyobs.append(newobs, ignore_index=True)

In [17]:
surveyobs

,ddecdt,dec,delta,dmagColor,dmagDetect,dmagTrail,dradt,fieldDec,fieldRA,filter,fiveSigmaDepth,magV,objId,observationStartMJD,phase,ra,seeingFwhmGeom,solarelon,velocity
0,-0.095931,-4.939275,1.561411,0.0,0.050512,0.058314,-0.117304,-5.689996,335.151362,y,22.194889,24.268074,S1000001a,59855.027905,13.455332,335.047920,0.392831,145.258206,0.151536
1,-0.093309,-5.036984,1.569287,0.0,0.024441,0.031789,-0.110926,-5.689996,335.151362,y,22.042133,24.292406,S1000001a,59856.062639,13.840847,334.937428,0.546925,144.120853,0.144952
2,0.102167,-34.439533,1.868874,0.0,0.014623,0.020346,-0.058082,-35.901259,327.493795,z,23.176332,15.093946,S1000009a,59853.087245,17.799468,327.652208,0.575949,127.415100,0.117523
3,0.102208,-34.439385,1.868890,0.0,0.014442,0.020124,-0.058070,-33.146604,327.373658,z,23.171840,15.093976,S1000009a,59853.088692,17.799758,327.652106,0.579741,127.413830,0.117553
4,0.106772,-34.234561,1.890874,0.0,0.026070,0.033590,-0.045917,-33.146604,327.373658,y,22.314798,15.134214,S1000009a,59855.043241,18.171668,327.545916,0.424285,125.726006,0.116227
5,0.106811,-34.234408,1.890891,0.0,0.025629,0.033105,-0.045920,-35.901259,327.493795,y,22.308882,15.134244,S1000009a,59855.044676,18.171944,327.545836,0.428145,125.724753,0.116264
